# FourCastNet v2 colab 教學版 

## 在colab中模擬虛擬環境 (請使用google中的colab)

In [ ]:
import sys
sys.path.append('/usr/local/envs/FCNV2/bin')  # 將 FCNV2 替換為你的虛擬環境名稱
!python -m ipykernel install --user --name=FCNV2 --display-name="Python (FCNV2)"


## 完成後須變更環境及GPU資源
#### 請沿以下路徑進行更改
#### [執行階段]>[變更執行階段類型]>[FCNV2] && [GPU]

## 需查看python 版本
##### 若不為3.10版需重新喚回

In [ ]:
# 查看版本是否為python 3.10
!python -V
# 鎖版本python3.10
# !sudo apt-get update
# !sudo apt-get install python3.10
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
# !sudo update-alternatives --config python3
# !python -V

Python 3.10.12


## download AI_global_model_for_education and FCNv2 weight

*  step1: 連到自己的google雲端(方便拿取輸出結果)
*  step2: git clone下載AI_global_model_for_education
*  step3: 下載FCNV2 模型權重變數
*  step4: 解壓縮並完成前置作業





In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Step 1: download AI_global_model_for_education and FCN weight
!git clone https://github.com/yungyun0721/AI_global_forecast_model_for_education.git
%cd /content/AI_global_forecast_model_for_education/FCNV2_for_copy
!wget 'https://api.ngc.nvidia.com/v2/models/nvidia/modulus/modulus_fcnv2_sm/versions/v0.2/files/fcnv2_sm.zip'
!unzip fcnv2_sm.zip
!mv fcnv2_sm weight
!rm -rf fcnv2_sm.zip

Mounted at /content/drive
Cloning into 'AI_global_forecast_model_for_education'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 286 (delta 115), reused 241 (delta 71), pack-reused 0 (from 0)
Receiving objects: 100% (286/286), 5.49 MiB | 7.73 MiB/s, done.
Resolving deltas: 100% (115/115), done.
/content/AI_global_forecast_model_for_education/FCNV2_for_copy
--2024-11-12 09:39:48--  https://api.ngc.nvidia.com/v2/models/nvidia/modulus/modulus_fcnv2_sm/versions/v0.2/files/fcnv2_sm.zip
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 52.40.141.214, 52.88.104.159
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|52.40.141.214|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://xfiles.ngc.nvidia.com/org/nvidia/team/modulus/models/modulus_fcnv2_sm/versions/v0.2/files/fcnv2_sm.zip?versionId=zQQpeu8EadI5maVMxR9C7cZMwPmdhy8d&Expires=1731490789&Signa

## 建置FCNV2需要的環境
*  step1: 更新pip模組
*  step2: 下載各種packages

In [ ]:
# install the environment
%cd /content/AI_global_forecast_model_for_education/FCNV2_for_copy
!pip install --upgrade pip
!pip install -r requirements.txt
!pip install hvplot

/content/AI_global_forecast_model_for_education/FCNV2_for_copy


## 下載initial 資料
#### 此次選用NCEP GFS的初始場 (僅有過去10天時間資料)

In [ ]:
# download the initial condition
%cd /content/AI_global_forecast_model_for_education/FCNV2_for_copy
!python download_ncep.py --scheduled-time 2024111000

/content/AI_global_forecast_model_for_education/FCNV2_for_copy
http://nomads.ncep.noaa.gov:80/dods/gfs_0p25/gfs20241110/gfs_0p25_00z
/usr/local/lib/python3.10/dist-packages/xarray/coding/times.py:187: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  ref_date = _ensure_padded_year(ref_date)
Done


## FCNV2資料展示

In [ ]:
import hvplot.xarray
import numpy as np
import xarray as xr
IC_data = np.load('/content/AI_global_forecast_model_for_education/FCNV2_for_copy/input_data/inital_condition.npy')
IC_flip = np.flip(IC_data, axis=1)
# variables order (73 variables)
ordering = [ "10u",  "10v", "100u", "100v",  "2t",  "sp",  "msl", "tcwv",
        "u50",  "u100", "u150", "u200", "u250", "u300", "u400", "u500", "u600", "u700", "u850", "u925", "u1000",
        "v50",  "v100", "v150", "v200", "v250", "v300", "v400", "v500", "v600", "v700", "v850", "v925", "v1000",
        "z50",  "z100", "z150", "z200", "z250", "z300", "z400", "z500", "z600", "z700", "z850", "z925", "z1000",
        "t50",  "t100", "t150", "t200", "t250", "t300", "t400", "t500", "t600", "t700", "t850", "t925", "t1000",
        "r50",  "r100", "r150", "r200", "r250", "r300", "r400", "r500", "r600", "r700", "r850", "r925", "r1000"]
index_ordering = 18

# lon lat 850 hpa windspeed
IC_for_plot = np.sqrt(IC_flip[index_ordering,:,:]**2+IC_flip[index_ordering+13,:,:]**2)
data = xr.DataArray(IC_for_plot, dims=['lat', 'lon'], coords={'lat': np.linspace(-90, 90, 721), 'lon': np.linspace(0,359.75,1440)})
plot = data.hvplot.image(x='lon', y='lat', cmap='jet', title='850 hpa wind speed')
plot

# lat_grid lon_grid windspeed
IC_for_find = np.sqrt(IC_data[index_ordering,:,:]**2+IC_data[index_ordering+13,:,:]**2)
data = xr.DataArray(IC_for_find, dims=['lat_grid', 'lon_grid'], coords={'lat_grid': np.linspace( 0, 720, 721), 'lon_grid': np.linspace(0,1439,1440)})
plot = data.hvplot.image(x='lon_grid', y='lat_grid', cmap='rainbow', title='850 hpa wind speed')
plot



Output hidden; open in https://colab.research.google.com to view.

##### 改 TC 風速 (可跳過)

In [ ]:
IC_data = np.load('/content/AI_global_forecast_model_for_education/FCNV2_for_copy/input_data/inital_condition.npy')
IC_data_tmp = IC_data.copy()
change_index = [i for i in range(8,34)]
lat_min = 296-20
lat_max = 296+20
lon_min = 624-20
lon_max = 624+20
IC_data[change_index,lat_min:lat_max,lon_min:lon_max] = (IC_data_tmp[change_index,lat_min:lat_max,lon_min:lon_max])*2
np.save('./input_data/inital_condition_double_wind.npy', IC_data.astype(np.float32))



In [ ]:
## check TC wind
IC_data = np.load('/content/AI_global_forecast_model_for_education/FCNV2_for_copy/input_data/inital_condition_double_wind.npy')
# lat_grid lon_grid windspeed
IC_for_find = np.sqrt(IC_data[index_ordering,:,:]**2+IC_data[index_ordering+13,:,:]**2)
data = xr.DataArray(IC_for_find, dims=['lat_grid', 'lon_grid'], coords={'lat_grid': np.linspace( 0, 720, 721), 'lon_grid': np.linspace(0,1439,1440)})
plot = data.hvplot.image(x='lon_grid', y='lat_grid', cmap='rainbow', title='850 hpa wind speed')
plot

Output hidden; open in https://colab.research.google.com to view.

## FCNV2預報
#### 目前使用GPU，預報240小時(每6小時一報)

In [ ]:
# FCNV2 predicting
%cd /content/AI_global_forecast_model_for_education/FCNV2_for_copy
!python main.py --input_data input_data/inital_condition.npy --output_folder output_data --fore_hr 120
# !python main.py --input_data input_data/inital_condition.npy --output_folder output_data --fore_hr 120 --device cuda

# !python main.py --input_data input_data/inital_condition_double_wind.npy --output_folder output_data_double_wind --fore_hr 120
# !python main.py --input_data input_data/inital_condition_double_wind.npy --output_folder output_data_double_wind_v2 --fore_hr 120 --device cuda

/content/AI_global_forecast_model_for_education/FCNV2_for_copy
/content/AI_global_forecast_model_for_education/FCNV2_for_copy/modules/inference_helper.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

## 預報完成後，查看結果
#### 進入plot底下，把850hPa的風場畫出

In [1]:
%cd /content/AI_global_forecast_model_for_education/FCNV2_for_copy/plot/
# !python plot850.py
!python plot850.py --data_source_file ../output_data --save_folder plot_figure

/content/AI_global_forecast_model_for_education/FCNV2_for_copy/plot
files count: 21
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


## 放到google雲端中方便下載

In [ ]:
%cd /content/AI_global_forecast_model_for_education/FCNV2_for_copy
!cp -rf plot /content/drive/MyDrive/AI_global_model/

/content/AI_global_forecast_model_for_education/FCNV2_for_copy
